In [60]:
import os
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist
import time
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import sys
sys.path.append(r'C:\Users\tacuevas\Documents\GitHub\concorde')
from mda import anti_neighbors_vectorized, fnorm, fnorm_inv, normalize_df, unnormalize_df
from plots import scatter_interpolate
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
plt.rcParams.update({'font.size': 14})
from matplotlib.offsetbox import AnchoredText
from itertools import combinations, permutations

In [33]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

#### Check new version of anti neighbors

In [2]:
df = pd.read_pickle(r'..\data\STORM\processed\STORM_NA_R2_In_pnt.pkl')
df.index = range(len(df))
df = df.drop(['year', 'month', 'yr_tc_number', 'tc_id', 'max_cat'], axis = 1)
df = df.fillna(0)
df.head()

,time_step_at_min_press,time_step_at_min_distNC,lat_at_min_press,lon_at_min_press,lat_at_min_distNC,lon_at_min_distNC,min_press_at_min_distNC,min_press_at_min_press,max_ws_at_min_distNC,max_ws_at_min_press,rad_to_max_ws_at_min_distNC,rad_to_max_ws_at_min_press,landfall,distNC_at_min_distNC,distNC_at_min_press,trajectory_at_min_distNC,trajectory_at_min_press,mean_trajectory
0,25.0,33.0,29.6,-76.2,30.5,-79.3,977.2,953.7,36.800000,36.800000,74.080002,74.080002,1.0,571.303714,623.563360,284.036243,303.690068,306.734075
1,20.0,49.0,17.2,-61.5,33.2,-68.2,954.5,939.2,46.600000,46.600000,74.080002,74.080002,0.0,822.784684,2505.942098,21.801409,284.036243,347.184714
2,4.0,8.0,25.9,-83.4,27.4,-82.8,1000.5,993.2,16.500000,16.500000,55.560001,27.780001,0.0,1035.895766,1209.183408,0.000000,18.434949,19.242162
3,4.0,9.0,25.9,-83.4,27.7,-82.5,1000.5,993.2,16.525852,16.525852,37.040001,37.040001,1.0,992.030799,1209.183408,0.000000,18.434949,22.104144
4,2.0,3.0,29.1,-76.9,29.2,-77.2,1003.8,1000.4,15.800000,15.800000,46.299999,46.299999,0.0,666.410514,676.494047,0.000000,288.434949,276.144983


In [3]:
index = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]

In [19]:
# columns = ['lat_at_min_distNC', 'lon_at_min_distNC', 'min_press_at_min_press', 'max_ws_at_min_press', 'rad_to_max_ws_at_min_press',
#           'mean_trajectory']
# index = [1, 1, 1, 1, 1, 0]
# df = df.loc[:, columns]

###### WIthout definitive function

In [4]:
norm = normalize_df(df, index)

In [5]:
norm.head()

,time_step_at_min_press,time_step_at_min_distNC,lat_at_min_press,lon_at_min_press,lat_at_min_distNC,lon_at_min_distNC,min_press_at_min_distNC,min_press_at_min_press,max_ws_at_min_distNC,max_ws_at_min_press,rad_to_max_ws_at_min_distNC,rad_to_max_ws_at_min_press,landfall,distNC_at_min_distNC,distNC_at_min_press,trajectory_at_min_distNC,trajectory_at_min_press,mean_trajectory
0,0.112613,0.141026,0.428302,0.232344,0.404669,0.225806,0.679558,0.581954,0.425664,0.425664,0.148936,0.304348,1.0,0.245419,0.097927,1.577979,1.687167,1.704078
1,0.090090,0.209402,0.194340,0.382805,0.509728,0.673387,0.470534,0.461921,0.573007,0.573007,0.148936,0.304348,0.0,0.353449,0.393545,0.121119,1.577979,1.928804
2,0.018018,0.034188,0.358491,0.158649,0.284047,0.084677,0.894107,0.908940,0.120454,0.120454,0.106383,0.086957,0.0,0.444996,0.189896,0.000000,0.102416,0.106901
3,0.018018,0.038462,0.358491,0.158649,0.295720,0.096774,0.894107,0.908940,0.120843,0.120843,0.063830,0.130435,1.0,0.426153,0.189896,0.000000,0.102416,0.122801
4,0.009009,0.012821,0.418868,0.225179,0.354086,0.310484,0.924494,0.968543,0.109930,0.109930,0.085106,0.173913,0.0,0.286274,0.106240,0.000000,1.602416,1.534139


In [6]:
a, b = [], []
for i, j in enumerate(index):
    if j == 0:
        b.append(i)
    elif j == 1:
        a.append(i)

In [7]:
for ib, bi in enumerate(b):
    aux = norm.iloc[:, bi].values
    m1 = np.broadcast_to(aux, (len(norm), len(norm)))
    if ib == 0:
        mdir = np.minimum(np.abs(m1 - m1.T), 2 - np.abs(m1 - m1.T))
    else:
        mdir = mdir + np.minimum(np.abs(m1 - m1.T), 2 - np.abs(m1 - m1.T))

aux = norm.iloc[:, a].values
msca = cdist(aux, aux, metric = 'sqeuclidean')

m = msca + mdir
del msca
del mdir

In [8]:
t0 = time.time()

k = 1000

points = [tuple(x) for x in norm.values]
dummy_index = list(df.index.values)
remaining_points = points[:]
solution_set = []
index_solution_set = []

tree = cKDTree(norm.values)
meanvalue = list(norm.mean())
ix_meanvalue = tree.query(meanvalue)[1]

solution_set.append(points[ix_meanvalue])
index_solution_set.append(dummy_index[ix_meanvalue])

# while len(solution_set) < k:
while len(solution_set) < k:
    t0 = time.time()
    if len(solution_set) == 1:
        ## iter 1, easy one
        distances = m[index_solution_set[-1], :]
        newSolIx = distances.argmax()
        solution_set.append(points[newSolIx])
        index_solution_set.append(newSolIx)
    else:
        ## iter 2, starts to get complicated
        distances = np.minimum(distances, m[index_solution_set[-1], :])
        newSolIx = distances.argmax()
        solution_set.append(points[newSolIx])
        index_solution_set.append(newSolIx)
#     print(f'Iter {time.time() - t0:0.4f}: s')
        
normclusters = pd.DataFrame(columns = df.columns, data = solution_set)
clusters_wf = unnormalize_df(df, normclusters, index)
clusters_wf['index_cluster'] = index_solution_set

In [9]:
clusters_wf.index_cluster.nunique()

1000

###### Using function

In [10]:
clusters = anti_neighbors_vectorized(df, 1000, index)

100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [01:16<00:00, 13.13it/s]


In [11]:
clusters.iloc[:, :-1].equals(clusters_wf)

True

#### MDA

In [2]:
df = pd.read_pickle(r'..\data\STORM\processed\STORM_NA_R2_In_pnt.pkl')
df.index = range(len(df))
df = df.drop(['year', 'month', 'yr_tc_number', 'tc_id', 'max_cat'], axis = 1)
df = df.fillna(0)
df.head()

,time_step_at_min_press,time_step_at_min_distNC,lat_at_min_press,lon_at_min_press,lat_at_min_distNC,lon_at_min_distNC,min_press_at_min_distNC,min_press_at_min_press,max_ws_at_min_distNC,max_ws_at_min_press,rad_to_max_ws_at_min_distNC,rad_to_max_ws_at_min_press,landfall,distNC_at_min_distNC,distNC_at_min_press,trajectory_at_min_distNC,trajectory_at_min_press,mean_trajectory
0,25.0,33.0,29.6,-76.2,30.5,-79.3,977.2,953.7,36.800000,36.800000,74.080002,74.080002,1.0,571.303714,623.563360,284.036243,303.690068,306.734075
1,20.0,49.0,17.2,-61.5,33.2,-68.2,954.5,939.2,46.600000,46.600000,74.080002,74.080002,0.0,822.784684,2505.942098,21.801409,284.036243,347.184714
2,4.0,8.0,25.9,-83.4,27.4,-82.8,1000.5,993.2,16.500000,16.500000,55.560001,27.780001,0.0,1035.895766,1209.183408,0.000000,18.434949,19.242162
3,4.0,9.0,25.9,-83.4,27.7,-82.5,1000.5,993.2,16.525852,16.525852,37.040001,37.040001,1.0,992.030799,1209.183408,0.000000,18.434949,22.104144
4,2.0,3.0,29.1,-76.9,29.2,-77.2,1003.8,1000.4,15.800000,15.800000,46.299999,46.299999,0.0,666.410514,676.494047,0.000000,288.434949,276.144983


In [35]:
index = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]

In [36]:
cl0 = anti_neighbors_vectorized(df, 1000, index)

100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [01:14<00:00, 13.38it/s]


In [43]:
print(f'min weight: {cl0.weight.min()*100:0.5f}')
print(f'max weight: {cl0.weight.max()*100:0.5f}')

min weight: 0.00309
max weight: 1.56825


In [38]:
cl1 = anti_neighbors_vectorized(df, 750, index)

100%|████████████████████████████████████████████████████████████████████████████████| 749/749 [00:37<00:00, 19.77it/s]


In [44]:
print(f'min weight: {cl1.weight.min()*100:0.5f}')
print(f'max weight: {cl1.weight.max()*100:0.5f}')

min weight: 0.00309
max weight: 1.83736


In [39]:
cl2 = anti_neighbors_vectorized(df, 500, index)

100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [00:15<00:00, 31.30it/s]


In [45]:
print(f'min weight: {cl2.weight.min()*100:0.5f}')
print(f'max weight: {cl2.weight.max()*100:0.5f}')

min weight: 0.00309
max weight: 1.91469


In [71]:
cl0.to_pickle((r'..\data\STORM\processed\MDA_output\STORM_NA_R2_In_pnt_Clusters_MDAvec_1000_allvars.pkl'))
cl1.to_pickle((r'..\data\STORM\processed\MDA_output\STORM_NA_R2_In_pnt_Clusters_MDAvec_0750_allvars.pkl'))
cl2.to_pickle((r'..\data\STORM\processed\MDA_output\STORM_NA_R2_In_pnt_Clusters_MDAvec_0500_allvars.pkl'))

#### Plot clusters

In [57]:
perm = list(permutations(df.columns, 2))

In [72]:
pathout = Path(r'..\data\STORM\processed\MDA_output\PLOTS_STORM_NA_R2_In_pnt_Clusters_MDAvec_allvars')

In [82]:
for p in tqdm(perm):
    xcol = p[0]
    ycol = p[1]
    
    x = df.loc[:, xcol]
    y = df.loc[:, ycol]
    z = scatter_interpolate(x,y)
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]
    cm = plt.cm.get_cmap('cividis')

    fig, ax = plt.subplots(figsize = (6, 4))
    ax.scatter(x, y, c = z, cmap = cm, alpha = 0.5)
    ax.scatter(cl0.loc[:, xcol], cl0.loc[:, ycol], c = 'r', s = 5)
    ax.scatter(cl1.loc[:, xcol], cl1.loc[:, ycol], c = 'orange', s = 5)
    ax.scatter(cl2.loc[:, xcol], cl2.loc[:, ycol], c = 'magenta', s = 5)
    
    ax.set_xlabel(xcol)
    ax.set_ylabel(ycol)
    
    text = AnchoredText(nb_name, 'upper right',frameon = False, borderpad = -2, prop=dict(fontsize = 'xx-small', alpha = 0.4))
    ax.add_artist(text)
    
    fig.savefig(pathout/f'{xcol}_VS_{ycol}.png', dpi = 100, bbox_inches = 'tight')
    plt.close()

100%|████████████████████████████████████████████████████████████████████████████████| 306/306 [04:16<00:00,  1.19it/s]


In [80]:
cl2.iloc[:, :-1].equals(cl0.iloc[:500, :-1])

True